# Formulas

$$d^{(j)}(z,t) = exp \left( -\sum_{n=1}^\infty \frac{z^n}{n} \sum_{|\underline i|=n} \frac {p_{\underline i}}{1-\lambda_{\underline i}^{-2}} \left(\prod_{l=0}^{n-1} \frac {\| A_j x_{\sigma^l \underline i} \|}{\|x_{\sigma^l \underline i}\|}\right)^t \right)$$

$$d^{(j)}(z,0) = exp \left( -\sum_{n=1}^\infty \frac{z^n}{n} \sum_{|\underline i|=n} \frac {p_{\underline i}}{1-\lambda_{\underline i}^{-2}} \right) = \color{green}{exp(-S_0)}$$

$$ \frac{\partial}{\partial t} d^{(j)}(z,t)_{|(z,0)} = \left(-\sum_{n=1}^\infty \frac{z^n}{n} \sum_{|\underline i|=n} \frac {p_{\underline i}}{1-\lambda_{\underline i}^{-2}} \log \left(\prod_{l=0}^{n-1} \frac {\| A_j x_{\sigma^l \underline i} \|}{\|x_{\sigma^l \underline i}\|}\right) \right) \cdot exp \left( -\sum_{n=1}^\infty \frac{z^n}{n} \sum_{|\underline i|=n} \frac {p_{\underline i}}{1-\lambda_{\underline i}^{-2}} \right) = \color{green}{-S_1 \exp(-S_0)}$$

$$x_{i_2, \dots, i_n, i_1} = A_{i_1}^{-1} x_{i_1, \dots, i_n}$$

In [23]:
import itertools
def terms(A,p,N):
    Ainv = [M.inverse() for M in A]
    R.<z> = CBF['z']
    k = len(A)

    S0,S1 = 0,[0]*k

    for n in range(1,N):
        cn_S0 = 0
        cn_S1 = [0]*k
        
        for i in itertools.product(range(k), repeat=n):
            M = prod(A[s] for s in i)
            l, x = perr_frob(M)
            scalar = prod(p[s] for s in i)/(1-l^(-2))

            P = [1]*k
            for l in range(n):
                for j in range(k):
                    P[j] *= CBF((A[j]*x).norm(1)/x.norm(1))
                x = Ainv[i[l]]*x
                
            # Use Arb polynomials
            cn_S0 += scalar
            for j in range(k):
                cn_S1[j] += scalar*log(P[j])

        for j in range(k):
            S1[j] += z^n/n*cn_S1[j]
        S0 += z^n/n*cn_S0
    
    an = (-S0)._exp_series(N)
    cn = [f1._mul_trunc_(-S1[j],N) for j in range(k)]
    
    return (an, cn)

def perr_frob(M):
    ev = M.eigenvectors_right()
    val, pos = 0, -1
    for i in range(len(ev)):
        v = ev[i]
        if v[0] > val :
            val = v[0]
            pos = i
    return(ev[pos][0], ev[pos][1][0])

$$d^{(j)}(z,0) = 1 + \sum_{n=1}^\infty a_n^{(j)}(0) z^n  = 1 + \sum_{n=1}^\infty a_n(0) z^n$$

$$\frac{d}{dz} d^{(j)}(z,0) = \sum_{n=1}^\infty n a_n^{(j)}(0) z^{n-1}$$
$$\frac{d}{dz} d^{(j)}(1,0) = \sum_{n=1}^\infty b_n^{(j)}$$

$$ \frac{\partial}{\partial t} d^{(j)}(z,t)_{|(z,0)} = \sum_{n=1}^\infty \frac{\partial a_n^{(j)}}{\partial t}(0) z^{n}$$

$$ \frac{\partial}{\partial t} d^{(j)}(z,t)_{|(1,0)} = \sum_{n=1}^\infty c_n^{(j)}$$

$$\lambda_n = \sum_{j=1}^k \frac{\sum_{i=1}^n c_i^{(j)}}{\sum_{i=1}^n b_i^{(j)}}$$

In [24]:
prec = 256
CBF = ComplexBallField(prec)

A = [
    matrix([[2,1],[1,1]]),
    matrix([[3,1],[2,1]])
]

p = [1/2,1/2]

an, cn = terms(A,p,5)
sum(cn[j].subs(z=1) for j in range(2))/(2*an.differentiate().subs(z=1))
# Where does the factor 2 come from ?

[1.14331103505173208520461882520715612957215148442230056673747095506825521805 +/- 7.07e-75]

In [25]:
an, cn = terms(A,p,10)
sum(cn[j].subs(z=1) for j in range(2))/(2*an.differentiate().subs(z=1))

[1.1433110351029492458432518536555882994025847950806845414580494534622463138 +/- 5.87e-74]

In [ ]:
1.1433110351029492458432518536555882994025